In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
data = pd.read_csv('data.csv')
final = pd.read_csv('test.csv')

In [ ]:
data.head()

,message,fingers,tail,species
0,pluvia arbor aquos,4,no,Aquari
1,cosmix xeno nebuz odbitaz,5,yes,Zorblax
2,solarix glixx novum galaxum quasar,5,yes,Zorblax
3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian
4,mermax drakos lorix epikoz deftax,4,no,Faerix


In [4]:
data['tail'] = data['tail'].map({'yes': 1, 'no': 0})
final['tail'] = final['tail'].map({'yes': 1, 'no': 0})

In [ ]:
def get_unique_words(messages):
    all_words = []
    for message in messages:
        words = message.split()  # Tokenize by splitting on whitespace
        all_words.extend(words)  # Add words to the list

    unique_words = set(all_words)  # Get unique words using a set
    return unique_words

# Get unique words from the 'message' column
unique_words = get_unique_words(data['message'])

# Count the total number of unique words
total_unique_words = len(unique_words)

print(f'Total number of unique words: {total_unique_words}')
# Was earlier trying to make patterns on basis of words present in the language instead of characters so did this
# but it did not work as there were lot of unknown words in test data hence used character approach

Total number of unique words: 636


In [5]:
data['message_length'] = data['message'].apply(lambda x: len(x.split()))
# made a new feature called message lenght that counts number of words in the message

In [6]:
final['message_length'] = final['message'].apply(lambda x: len(x.split()))


In [ ]:
#data['unique_chars'] = data['message'].apply(lambda x: len(set(x)))  # tried to make a new feature based on number of characters but it did not work so not used in training
#final['unique_chars'] = final['message'].apply(lambda x: len(set(x)))

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialized the vectorizer to work at the character level as word level was not orking due to lot of unknown words in test data
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 7), max_features=900)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Encoding the species column
y = label_encoder.fit_transform(data['species'])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[['message','tail','fingers','message_length']], y, test_size=0.2, random_state=2, stratify=y)
X_train_char = char_vectorizer.fit_transform(X_train['message'])
X_train_combined = pd.concat([pd.DataFrame(X_train_char.toarray()), X_train[['fingers', 'tail','message_length']].reset_index(drop=True)], axis=1)
#combined all data to make final training set

X_test1_char = char_vectorizer.transform(X_test['message'])
X_test1_combined = pd.concat([pd.DataFrame(X_test1_char.toarray()), X_test[['fingers', 'tail','message_length']].reset_index(drop=True)], axis=1)




In [ ]:
X_train_combined.head()

,0,1,2,3,4,5,6,7,8,9,...,893,894,895,896,897,898,899,fingers,tail,message_length
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,no,8
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,yes,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,3,yes,4
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,3,no,7
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,no,3


In [ ]:
from xgboost import XGBClassifier

# # Initialized the XGBoost classifier algorithm
xgb_model = XGBClassifier(n_estimators=200, max_depth=4, use_label_encoder=False, eval_metric='mlogloss', random_state=2)
# random state set for consistent results
# Train the XGBoost model
xgb_model.fit(X_train_combined, y_train)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:39:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
from sklearn.metrics import accuracy_score

# Make predictions on the test set
y_pred = xgb_model.predict(X_test1_combined)

# Calculating accuracy for the model on the test set and then tuning the hyperparameters to get max possible accuracy
# we also adjusted the max_features and ngram_range using the accuracy metric to get max accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 78.00%


In [9]:
 #after gatting the best possible model we will train the model on the entire train set and not just 80% of it to get best possible result


X_trainfull_char = char_vectorizer.fit_transform(data['message'])
X_trainfull_combined = pd.concat([pd.DataFrame(X_trainfull_char.toarray()), data[['fingers', 'tail','message_length']].reset_index(drop=True)], axis=1)

X_test_char = char_vectorizer.transform(final['message'])
X_test_combined = pd.concat([pd.DataFrame(X_test_char.toarray()), final[['fingers', 'tail','message_length']].reset_index(drop=True)], axis=1)
# Combined with numerical features (fingers, tail and message lenght)

In [10]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=200, max_depth=4, use_label_encoder=False, eval_metric='mlogloss', random_state=2)

# Train the XGBoost model on entire training set
xgb_model.fit(X_trainfull_combined, y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:55:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [11]:


# Make predictions on the test set
y_predfinal = xgb_model.predict(X_test_combined)



In [16]:
Species_pred = label_encoder.inverse_transform(y_predfinal) # decoding the species names

# Print the decoded species names
print(Species_pred)

['Aquari' 'Sentire' 'Florian' 'Faerix' 'Mythron' 'Mythron' 'Faerix'
 'Faerix' 'Emotivor' 'Florian' 'Mythron' 'Emotivor' 'Zorblax' 'Quixnar'
 'Mythron' 'Aquari' 'Florian' 'Emotivor' 'Aquari' 'Florian' 'Zorblax'
 'Emotivor' 'Quixnar' 'Nexoon' 'Mythron' 'Nexoon' 'Florian' 'Sentire'
 'Emotivor' 'Mythron' 'Cybex' 'Emotivor' 'Mythron' 'Sentire' 'Faerix'
 'Emotivor' 'Aquari' 'Florian' 'Aquari' 'Zorblax' 'Emotivor' 'Quixnar'
 'Mythron' 'Nexoon' 'Aquari' 'Emotivor' 'Zorblax' 'Quixnar' 'Zorblax'
 'Florian' 'Emotivor' 'Zorblax' 'Sentire' 'Quixnar' 'Emotivor' 'Emotivor'
 'Sentire' 'Emotivor' 'Mythron' 'Cybex' 'Nexoon' 'Cybex' 'Mythron'
 'Sentire' 'Sentire' 'Mythron' 'Sentire' 'Faerix' 'Nexoon' 'Quixnar'
 'Quixnar' 'Emotivor' 'Sentire' 'Zorblax' 'Emotivor' 'Sentire' 'Cybex'
 'Mythron' 'Nexoon' 'Emotivor' 'Emotivor' 'Cybex' 'Emotivor' 'Florian'
 'Nexoon' 'Sentire' 'Faerix' 'Quixnar' 'Nexoon' 'Faerix' 'Sentire'
 'Faerix' 'Nexoon' 'Quixnar' 'Sentire' 'Florian' 'Mythron' 'Nexoon'
 'Emotivor' 'Mythron' 

In [17]:
len(Species_pred) # verifying lenght of species


299

In [18]:
# Adding a column named 'Species' to the test DataFrame
result_df = pd.DataFrame({
    'Species': Species_pred  # Replace with your actual predictions array
})

# Saving the updated DataFrame to a new CSV file
result_df.to_csv('result.csv', index=False)


In [19]:
from google.colab import files

# Download the result.csv file
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>